In [1]:
import pandas as pd
import xmltodict

In [2]:
xml_path = "F:/Workspace/Python/Estagio/dados.xml"
json_path = "F:/Workspace/Python/Estagio/dados.json"
excel_path = "F:/Workspace/Python/Estagio/data.xlsx"

In [3]:
df = pd.read_json(json_path)

df['dia'] = df['dia'].fillna(0).astype(int)
df['valor'] = df['valor'].fillna(0).astype(float).round(4)
df.columns = ['dia', 'json valor']

In [4]:
# adicionada a tag <rows> manualmente em dados.xmlp para funcionar

with open(xml_path, "rb") as data:
  xml = xmltodict.parse(data)

rows = xml['rows']['row']
valores = [row['valor'] for row in rows]
df['xml valor'] = valores
df['xml valor'] = df['xml valor'].fillna(0).astype(float).round(4)

In [5]:
# adicionar os dias da semana e cada semana
# o mês dos dados começa na quarta

dias_semana = ["quarta-feira",
               "quinta-feira",
               "sexta-feira",
               "sábado",
               "domingo",
               "segunda-feira",
               "terça-feira"]

dias_repetidos = (dias_semana * (30 // len(dias_semana)+1))[:30]

df['dia da semana'] = dias_repetidos
df['semana'] = (df['dia'] - 1) // 7 + 1

In [6]:
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='r+', if_sheet_exists='overlay') as writer:
  df.to_excel(writer, sheet_name='xml', index=False)

In [7]:
zeros_json = 0
val_json = 0

zeros_xml = 0
val_xml = 0

for v in df['json valor']:
  if v == 0:
    zeros_json += 1
  else:
    val_json += 1


for v in df['xml valor']:
  if v == 0:
    zeros_xml += 1
  else:
    val_xml += 1

In [8]:
# insights

insights = {
  "media": [df['json valor'].mean().round(4), df['xml valor'].mean().round(4)],
  "max": [df['json valor'].max().round(4), df['xml valor'].max().round(4)],
  "dias_trabalhados": [val_json, val_xml],
  "dias_folga": [zeros_json, zeros_xml],
  "total": [df['json valor'].sum().round(4), df['xml valor'].sum().round(4)],
  "menor_faturamento": [df[df['json valor'] != 0]['json valor'].min().round(4),
                       df[df['xml valor'] != 0]['xml valor'].min().round(4)],
  "maior_faturamento": [df['json valor'].max().round(4),
                        df['xml valor'].max().round(4)],
}


# saldos
df['saldo_diario'] =  df['xml valor'] - df['json valor']

soma_semanal_json = df.groupby('semana')['json valor'].sum().reset_index()
soma_semanal_xml = df.groupby('semana')['xml valor'].sum().reset_index()


df_saldo_semanal = pd.merge(soma_semanal_json, soma_semanal_xml, on='semana')
df_saldo_semanal['saldo_semanal'] = (soma_semanal_xml['xml valor'] - soma_semanal_json['json valor'])

saldo_mensal = soma_semanal_xml['xml valor'].sum() - soma_semanal_json['json valor'].sum()

In [9]:
# saídas

for key, value in insights.items():
  print(f"{key}: json: {value[0]} - xml: {value[1]}")

print("----------")
print(df_saldo_semanal)
print("----------")
print(f"Saldo mensal: {saldo_mensal}")

media: json: 14605.7591 - xml: 16743.2136
max: json: 48924.2448 - xml: 48275.2994
dias_trabalhados: json: 21 - xml: 21
dias_folga: json: 9 - xml: 9
total: json: 438172.7735 - xml: 502296.4079
menor_faturamento: json: 373.7838 - xml: 3071.3283
maior_faturamento: json: 48924.2448 - xml: 48275.2994
----------
   semana   json valor    xml valor  saldo_semanal
0       1   99594.1108  124411.3838     24817.2730
1       2  104553.1381  112705.0043      8151.8662
2       3  149072.6118  128254.9544    -20817.6574
3       4   63317.8078   86751.2820     23433.4742
4       5   21635.1050   50173.7834     28538.6784
----------
Saldo mensal: 64123.63440000004
